In [1]:
import mlrun

# Initialize mlrun environment and load the project
from os import path


project_name = "widsdb2"
project_path = "conf"

widsdb2_proj = mlrun.projects.load_project(project_path,  clone=True)


#widsdb2_proj = mlrun.new_project(project_name,
#                            context=project_dir,
#                            init_git=True,
#                            user_project=False)


NameError: name 'project_path' is not defined

In [174]:
%nuclio cmd -c pip install cdqa

In [175]:
%nuclio config spec.build.baseImage = "mlrun/mlrun"
%nuclio config kind = "serving"

%nuclio: setting spec.build.baseImage to 'mlrun/mlrun'
%nuclio: setting kind to 'serving'


In [176]:
# nuclio: start-code

In [177]:
import os
from ast import literal_eval
from cdqa.pipeline import QAPipeline
from cdqa.utils.download import download_model
import joblib
import numpy as np
from cloudpickle import load
import mlrun

import sys
sys.path.append('/v3io/projects/widsdb2/util')



class QnAModel(mlrun.serving.V2ModelServer):
    
    def load(self):
        model_file, extra_data = self.get_model('.pkl')
        print(model_file)
        self.model = joblib.load(model_file, mmap_mode='r')
        #self.model = load(open(model_file, 'rb'))
     
        #model_file, extra_data = self.get_model('.joblib')
        #self.model = joblib.load(open(model_file, 'rb'))

    def predict(self, body):
        try:
            feats = np.asarray(body['inputs'])
            print(feats)
            query = body['inputs'][0]['query']
            print(query)
            prediction = self.model.predict(query, 3)
            print(prediction)
            return prediction
            #return prediction.tolist()
        except Exception as exc:
            raise Exception(f"Failed to predict {exc}")

In [178]:
# nuclio: end-code

In [179]:
import mlrun
from mlrun import code_to_function



serving_fn = mlrun.code_to_function('QnAserving',
                            description="QnAServing",
                            categories=['serving', 'ml'],
                            labels={'author': 'Aruna', 'framework': 'WQnAModel'})
serving_fn.spec.default_class = 'QnAModel'




In [180]:
#df = pd.read_csv('dm_data.csv')
#df

In [181]:
#tmodel=joblib.load('models/bert_qa.joblib')
tmodel = 'qa.pkl'
serving_fn.add_model('bertqa_model',model_path='/v3io/projects/widsdb2/util/qa.pkl')


In [182]:
widsdb2_proj.set_function(serving_fn)

In [183]:
my_data=[{'query': 'what are diabetes mellitus risks ?'}]

In [184]:
## Run the inference with the serving function
server = serving_fn.to_mock_server()
pred = server.test("/v2/models/bertqa_model/infer", body={"inputs": my_data})
#server.test("/v2/models/bertqa_model/infer", body={"inputs": my_data})
pred

/v3io/projects/widsdb2/util/qa.pkl
> 2021-07-05 14:43:59,843 [info] model bertqa_model was loaded
> 2021-07-05 14:43:59,845 [info] Loaded ['bertqa_model']
[{'query': 'what are diabetes mellitus risks ?'}]
what are diabetes mellitus risks ?
[('refractory to sulfonylureas', 's41574-020-00435-4', '218. Gerstein, H. C., Thorpe, K. E., Taylor, D. W.  & Haynes, R. B. The effectiveness of hydroxychloroquine in patients with type 2 diabetes mellitus who are refractory to sulfonylureas–a randomized trial. Diabetes Res. Clin. Pract. 55, 209–219 (2002).', 9.91593705015013), ('thromboembolism and bleeding', 's41574-020-00435-4', '157. Overvad, T. F. et\xa0al. Duration of diabetes mellitus and risk of thromboembolism and bleeding in atrial fibrillation: nationwide cohort study. Stroke 46, 2168–2174 (2015).', 9.8835626472094), ('comorbidities', 's41574-020-00435-4', '44. Cheema, A. K. et\xa0al. Integrated datasets of proteomic and metabolomic biomarkers to predict its impacts on comorbidities of typ

{'id': '7e8f09a067044269b4212a5563082220',
 'model_name': 'bertqa_model',
 'outputs': [('refractory to sulfonylureas',
   's41574-020-00435-4',
   '218. Gerstein, H. C., Thorpe, K. E., Taylor, D. W.  & Haynes, R. B. The effectiveness of hydroxychloroquine in patients with type 2 diabetes mellitus who are refractory to sulfonylureas–a randomized trial. Diabetes Res. Clin. Pract. 55, 209–219 (2002).',
   9.91593705015013),
  ('thromboembolism and bleeding',
   's41574-020-00435-4',
   '157. Overvad, T. F. et\xa0al. Duration of diabetes mellitus and risk of thromboembolism and bleeding in atrial fibrillation: nationwide cohort study. Stroke 46, 2168–2174 (2015).',
   9.8835626472094),
  ('comorbidities',
   's41574-020-00435-4',
   '44. Cheema, A. K. et\xa0al. Integrated datasets of proteomic and metabolomic biomarkers to predict its impacts on comorbidities of type 2 diabetes mellitus. Diabetes Metab. Syndr. Obes. 13, 2409–2431 (2020).',
   9.633038617049458)]}

In [185]:
pred['outputs']
pred['outputs']


[('refractory to sulfonylureas',
  's41574-020-00435-4',
  '218. Gerstein, H. C., Thorpe, K. E., Taylor, D. W.  & Haynes, R. B. The effectiveness of hydroxychloroquine in patients with type 2 diabetes mellitus who are refractory to sulfonylureas–a randomized trial. Diabetes Res. Clin. Pract. 55, 209–219 (2002).',
  9.91593705015013),
 ('thromboembolism and bleeding',
  's41574-020-00435-4',
  '157. Overvad, T. F. et\xa0al. Duration of diabetes mellitus and risk of thromboembolism and bleeding in atrial fibrillation: nationwide cohort study. Stroke 46, 2168–2174 (2015).',
  9.8835626472094),
 ('comorbidities',
  's41574-020-00435-4',
  '44. Cheema, A. K. et\xa0al. Integrated datasets of proteomic and metabolomic biomarkers to predict its impacts on comorbidities of type 2 diabetes mellitus. Diabetes Metab. Syndr. Obes. 13, 2409–2431 (2020).',
  9.633038617049458)]

In [186]:
serving_fn.apply(mlrun.platforms.auto_mount())
address = serving_fn.deploy()


> 2021-07-05 14:44:12,496 [info] Starting remote function deploy
2021-07-05 14:44:14  (info) Deploying function
2021-07-05 14:44:14  (info) Building
2021-07-05 14:44:14  (info) Staging files and preparing base images
2021-07-05 14:44:14  (info) Building processor image
2021-07-05 14:44:23  (info) Build complete
2021-07-05 14:44:57  (info) Function deploy complete
> 2021-07-05 14:44:58,901 [info] function deployed, address=default-tenant.app.mlops1.iguazio-c0.com:32046


In [ ]:
## Run the inference with the serving function
server = serving_fn.to_mock_server()
pred = server.test("/v2/models/bertqa_model/infer", body={"inputs": my_data})
#server.test("/v2/models/bertqa_model/infer", body={"inputs": my_data})
pred.outputs

In [172]:
address("/v2/models/bertqa_model/infer", body={"inputs": my_data})

TypeError: 'str' object is not callable